In [628]:
#第一題
import pandas as pd
import copy

sales = pd.read_excel('sales.xlsx') #讀入sales資料集，DataFrame格式


In [629]:
#第二題
sales.info() #用 DataFrame的info函數看欄位名稱跟每欄位的資料型態

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65535 entries, 0 to 65534
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       65535 non-null  int64  
 1   Product    65535 non-null  object 
 2   line       65535 non-null  object 
 3   Product.1  65535 non-null  object 
 4   type       65535 non-null  object 
 5   Product.2  65535 non-null  object 
 6   Order      65535 non-null  object 
 7   method     61083 non-null  object 
 8   type.1     46709 non-null  object 
 9   Retailer   32525 non-null  object 
 10  country    24135 non-null  object 
 11  Revenue    21494 non-null  object 
 12  Planned    20925 non-null  object 
 13  revenue    20645 non-null  object 
 14  Product.3  20588 non-null  object 
 15  cost       19224 non-null  float64
 16  Quantity   13663 non-null  float64
 17  Unit       7046 non-null   float64
 18  cost.1     2403 non-null   float64
 19  Unit.1     650 non-null    float64
 20  price 

In [630]:
#第三題
sales.head(1)

#此程式碼印出第一筆資料，可以看到與作業文件第三題的圖差異在於有些欄位的資料溢位了，像是Camping Equipment應該是屬於同一個欄位(Product line欄)，
#原始檔案的Camping Equipment拆成兩欄了，讓原本的欄位變成Product和line兩欄
#後面欄位也有很多都是錯的，實際上欄位數量應該只有15個，因此需要對原始資料集做處理(在第四題進行處理)


,Year,Product,line,Product.1,type,Product.2,Order,method,type.1,Retailer,...,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1
0,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,...,6.59,156672.57,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [631]:
#第四題
#4.1

#先看Product line欄位的溢位狀況
sales.groupby(['Product', 'line']).size()

Product         line       
Camping         Equipment      20112
Golf            Equipment       6615
Mountaineering  Equipment       9261
Outdoor         Protection      6615
Personal        Accessories    22932
dtype: int64

In [632]:
#再檢查 Product line欄的資料有沒有溢位到 Product.1欄
sales.groupby(['Product', 'line', 'Product.1']).size()

#檢查完發現 Product line欄的資料沒有溢位到 Product.1欄位，所以只要處理 Product跟 line兩欄的資料溢位問題


Product         line         Product.1 
Camping         Equipment    Cooking       5880
                             Lanterns      5292
                             Packs         2646
                             Sleeping      3087
                             Tents         3207
Golf            Equipment    Golf          1764
                             Irons         1764
                             Putters       1323
                             Woods         1764
Mountaineering  Equipment    Climbing      3087
                             Rope          1764
                             Safety        1764
                             Tools         2646
Outdoor         Protection   First         2205
                             Insect        2205
                             Sunscreen     2205
Personal        Accessories  Binoculars    2646
                             Eyewear       7056
                             Knives        3087
                             Navigation    4410


In [633]:
#把 Product跟 line兩欄合併成 Product line欄位
sales['Product line'] = sales['Product'] + " " + sales['line']

sales = sales.drop(['Product', 'line'], axis = 1) #把 Product和 line欄位刪除

sales.head()

,Year,Product.1,type,Product.2,Order,method,type.1,Retailer,country,Revenue,...,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line
0,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,States,315044,...,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
1,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,...,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
2,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
3,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
4,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,181120,235237,...,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment


In [634]:
#最後確認是否原本 Product和 line欄位全部都成功合併成 Product line欄
sales.groupby('Product line').size()

#跟前面資料筆數吻合，所以全部都正確合併了

Product line
Camping Equipment           20112
Golf Equipment               6615
Mountaineering Equipment     9261
Outdoor Protection           6615
Personal Accessories        22932
dtype: int64

In [635]:
#4.2
#只需保留：Climbing Accessories, Cooking Gear, First Aid, Golf Accessories, Insect Repellents, Sleeping Bags
#用 isin函數判斷 Product.1和 type欄位有沒有那六個種類的字，只保留有那些字的instance，以deepcopy複製保留那六個種類的 DataFrame
# Series的 operator要用 Bitwise operator!所以這裡的 and要用 &，不然會錯
#用deepcopy複製是為了避免改到原本 sales的資料
tmp1 = copy.deepcopy(sales[sales['Product.1'].isin(['Climbing', 'Cooking', 'First', 'Golf', 'Insect', 'Sleeping']) & sales['type'].isin(['Accessories', 'Gear', 'Aid', 'Repellents', 'Bags'])])


In [636]:
#確認 Product type溢位情況

#先檢查Product.1 和 type
tmp1.groupby(['Product.1', 'type']).size()

Product.1  type       
Climbing   Accessories    3087
Cooking    Gear           5880
First      Aid            2205
Golf       Accessories    1764
Insect     Repellents     2205
Sleeping   Bags           3087
dtype: int64

In [637]:
#再檢查 有沒有溢位到Product.2
tmp1.groupby(['Product.1', 'type', 'Product.2']).size()

#檢查發現沒有溢位到 Product.2的資料，所以把 tmp1的 Product.1和 type欄位合併成 Product type欄位就行了

Product.1  type         Product.2 
Climbing   Accessories  Firefly       1323
                        Granite       1764
Cooking    Gear         TrailChef     5880
First      Aid          Aloe           441
                        Calamine       441
                        Compact        441
                        Deluxe         441
                        Insect         441
Golf       Accessories  Course        1764
Insect     Repellents   BugShield     2205
Sleeping   Bags         Hibernator    3087
dtype: int64

In [638]:
#合併 Product.1和 type成 Product type
tmp1['Product type'] = tmp1['Product.1'] + " " + tmp1['type']
tmp1 = tmp1.drop(['Product.1', 'type'], axis = 1)

tmp1.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [639]:
#4.3

#先檢查 Product.2和 Order欄位
tmp1.groupby(['Product.2', 'Order']).size()

#發現資料溢位情形不同，因此接下來先處理溢位最多欄的資料

Product.2   Order       
Aloe        Relief           441
BugShield   Extreme          441
            Lotion           882
            Natural          441
            Spray            441
Calamine    Relief           441
Compact     Relief           441
Course      Pro             1764
Deluxe      Family           441
Firefly     Charger          441
            Climbing         441
            Rechargeable     441
Granite     Belay            441
            Carabiner        441
            Chalk            441
            Pulley           441
Hibernator  Camp             441
            E-mail            63
            Extreme          441
            Fax               63
            Lite             441
            Mail              63
            Pad              441
            Pillow           441
            Sales             63
            Self             441
            Special           63
            Telephone         63
            Web               63
Insect      Bite  

In [640]:
#排除 Order欄位是 E-mail, Fax, Mail, Sales, Special, Telephone, Web，並以deepcopy複製存成tmp2
tmp2 = copy.deepcopy(tmp1[~tmp1['Order'].isin(['E-mail', 'Fax', 'Mail', 'Sales', 'Special', 'Telephone', 'Web'])])
tmp2.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [641]:
#往下一個欄位： method檢查溢位情況
tmp2.groupby(['Product.2', 'Order', 'method']).size()

Product.2  Order     method   
Aloe       Relief    E-mail        63
                     Fax           63
                     Mail          63
                     Sales         63
                     Special       63
                                 ... 
TrailChef  Utensils  Sales         84
                     Special       84
                     Telephone     84
                     Web           84
           Water     Bag          588
Length: 144, dtype: int64

In [642]:
#一樣的動作，把 method欄位有 E-mail, Fax, Mail, Sales, Special, Telephone, Web的排除，並以deepcopy複製存成tmp3
tmp3 = copy.deepcopy(tmp2[~tmp2['method'].isin(['E-mail', 'Fax', 'Mail', 'Sales', 'Special', 'Telephone', 'Web'])])
tmp3.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [643]:
#檢查 type.1欄位溢位情況
tmp3.groupby(['Product.2', 'Order', 'method', 'type.1']).size()

Product.2  Order   method  type.1   
BugShield  Lotion  Lite    E-mail       63
                           Fax          63
                           Mail         63
                           Sales        63
                           Special      63
                                        ..
TrailChef  Water   Bag     Mail         84
                           Sales        84
                           Special      84
                           Telephone    84
                           Web          84
Length: 103, dtype: int64

In [644]:
#把 type.1欄位有 E-mail, Fax, Mail, Sales, Special, Telephone, Web的排除，並以deepcopy複製存成tmp4
tmp4 = copy.deepcopy(tmp3[~tmp3['type.1'].isin(['E-mail', 'Fax', 'Mail', 'Sales', 'Special', 'Telephone', 'Web'])])
tmp4.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
735,2004,TrailChef,Deluxe,Cook,Set,Telephone,United,States,865351,930482,...,254856.69,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
736,2004,TrailChef,Deluxe,Cook,Set,Telephone,Canada,67799.7,72902.6,47831.8,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
737,2004,TrailChef,Deluxe,Cook,Set,Telephone,Mexico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
738,2004,TrailChef,Deluxe,Cook,Set,Telephone,Brazil,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
739,2004,TrailChef,Deluxe,Cook,Set,Telephone,Japan,419827,451426,296183,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [645]:
#檢查 Retailer資料溢位情況
tmp4.groupby(['Product.2', 'Order', 'method', 'type.1', 'Retailer']).size()



Product.2   Order   method  type.1     Retailer 
Course      Pro     Golf    Bag        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
                            and        Tee          441
Deluxe      Family  Relief  Kit        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
Hibernator  Self    0       Inflating  Mat          441
TrailChef   Deluxe  Cook    Set        E-mail        84

In [646]:
#把 Retailer欄位有 E-mail, Fax, Mail, Sales, Special, Telephone, Web的排除，並以deepcopy複製存成tmp5
tmp5 = copy.deepcopy(tmp4[~tmp4['Retailer'].isin(['E-mail', 'Fax', 'Mail', 'Sales', 'Special', 'Telephone', 'Web'])])
tmp5.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2793,2004,Hibernator,Self,0,Inflating,Mat,Telephone,United,States,901517,...,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2794,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Canada,57049.5,61344.1,...,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2795,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Mexico,28879.8,31053.9,...,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2796,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Brazil,19647.8,21126.8,...,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2797,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Japan,423905,470263,...,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [647]:
#檢查 country資料溢位情況
tmp5.groupby(['Product.2', 'Order', 'method', 'type.1', 'Retailer', 'country']).size()

Product.2   Order  method  type.1     Retailer  country  
Course      Pro    Golf    and        Tee       Set          441
Hibernator  Self   0       Inflating  Mat       E-mail        63
                                                Fax           63
                                                Mail          63
                                                Sales         63
                                                Special       63
                                                Telephone     63
                                                Web           63
dtype: int64

In [648]:
#把 country欄位有 E-mail, Fax, Mail, Sales, Special, Telephone, Web的排除，並以deepcopy複製存成tmp6
tmp6 = copy.deepcopy(tmp5[~tmp5['country'].isin(['E-mail', 'Fax', 'Mail', 'Sales', 'Special', 'Telephone', 'Web'])])
tmp6.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
17493,2004,Course,Pro,Golf,and,Tee,Set,Telephone,United,States,...,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17494,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Canada,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17495,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Mexico,32224.2,...,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17496,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Brazil,10268.4,...,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17497,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Japan,32314.7,...,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories


In [649]:
#檢查 Revenue資料溢位情況
tmp6.groupby(['Product.2', 'Order', 'method', 'type.1', 'Retailer', 'country', 'Revenue']).size()

#可以看到原本屬於 Product欄的資料溢位最長到 country!

Product.2  Order  method  type.1  Retailer  country  Revenue  
Course     Pro    Golf    and     Tee       Set      E-mail       63
                                                     Fax          63
                                                     Mail         63
                                                     Sales        63
                                                     Special      63
                                                     Telephone    63
                                                     Web          63
dtype: int64

In [650]:
#把 Product.2 ~ country的資料併成一個欄位： Product
tmp6['Product'] = tmp6['Product.2'] + " " + tmp6['Order'] + " " + tmp6['method'] + " " + tmp6['type.1'] + " " + tmp6['Retailer'] + " " + tmp6['country']
#把 Product.2~country的欄位刪掉
tmp6 = tmp6.drop(['Product.2', 'Order', 'method', 'type.1', 'Retailer', 'country'], axis = 1)
tmp6.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17493,2004,Telephone,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17494,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17495,2004,Telephone,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17496,2004,Telephone,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17497,2004,Telephone,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [651]:
#再來處理原本屬於Order method type欄的資料溢位問題。
tmp6.groupby(['Revenue', 'Planned']).size()


Revenue  Planned    
E-mail   Australia      3
         Austria        3
         Belgium        3
         Brazil         3
         Canada         3
                       ..
Web      Singapore      3
         Spain          3
         Sweden         3
         Switzerland    3
         United         6
Length: 121, dtype: int64

In [652]:
#因為只有sales visit會溢位，所以先處理sales visit
tmp7 = copy.deepcopy(tmp6[tmp6['Planned'] == 'visit'])
tmp7.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17514,2004,Sales,visit,United,States,5819.70,6586.16,1733.2,619.0,2.80,10.64,4086.500000,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17515,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17516,2004,Sales,visit,Mexico,13497.6,14598.08,3841.60,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17517,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17518,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [653]:
#合併 Revenue和 Planned
tmp7['Order method type'] = tmp7['Revenue'] + " " + tmp7['Planned']
tmp7 = tmp7.drop(['Revenue', 'Planned'], axis = 1)
tmp7.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17514,2004,United,States,5819.70,6586.16,1733.2,619.0,2.80,10.64,4086.500000,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17515,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17516,2004,Mexico,13497.6,14598.08,3841.60,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17517,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17518,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


In [654]:
#再來處理原本屬於 Retailer country欄的資料溢位問題

#檢查 revenue和 Product.3
tmp7.groupby(['revenue', 'Product.3']).size()

revenue      Product.3
Australia    21183.33     1
             24114.16     1
Austria      0            3
China        27162.24     1
Denmark      6013.69      1
Finland      21185.75     1
France       0            1
             16307.46     1
             20542.05     1
Germany      3962.88      1
Italy        0            1
Japan        16448.11     1
Korea        8433.46      1
Mexico       13497.62     1
             23901.12     1
             30444        1
Netherlands  10970.16     1
Singapore    7422.67      1
Sweden       13058.36     1
Switzerland  12745.46     1
             28451.78     1
United       Kingdom      3
             States       3
dtype: int64

In [655]:
#檢查 revenue ~ cost
tmp7.groupby(['revenue', 'Product.3', 'cost']).size()

#可以看到會溢位的只有 United Kingdom和 United States，所以先處理這兩個

revenue      Product.3  cost    
Australia    21183.33   22195.04    1
             24114.16   25589.20    1
Austria      0          127.68      1
                        425.60      1
                        638.40      1
China        27162.24   28621.60    1
Denmark      6013.69    6266.96     1
Finland      21185.75   22897.28    1
France       0          723.52      1
             16307.46   16758.00    1
             20542.05   22354.64    1
Germany      3962.88    4085.76     1
Italy        0          851.20      1
Japan        16448.11   16779.28    1
Korea        8433.46    8788.64     1
Mexico       13497.62   14598.08    1
             23901.12   25184.88    1
             30444      33143.60    1
Netherlands  10970.16   11310.32    1
Singapore    7422.67    7735.28     1
Sweden       13058.36   13321.28    1
Switzerland  12745.46   13566.00    1
             28451.78   30600.64    1
United       Kingdom    8750.77     1
                        27987.84    1
             Stat

In [656]:
#保留 Product.3有 Kingdom 和 States的資料，並以deepcopy複製存成 tmp8
tmp8 = copy.deepcopy(tmp7[tmp7['Product.3'].isin(['Kingdom', 'States'])])
tmp8.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17514,2004,United,States,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17529,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
38682,2005,United,States,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
38697,2005,United,Kingdom,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
59850,2006,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


In [657]:
#合併 revenue和 Product.3
tmp8['Retailer country'] = tmp8['revenue'] + " " + tmp8['Product.3']
tmp8 = tmp8.drop(['revenue', 'Product.3'], axis = 1)
tmp8.head()

,Year,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
17514,2004,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States
17529,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom
38682,2005,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States
38697,2005,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom
59850,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States


In [658]:
#再來因為資料型態都是數字型態，所以不會有資料溢位問題，修改欄位名稱即可
tmp8['Revenue'] = tmp8['cost']
tmp8 = tmp8.drop('cost', axis = 1)

tmp8['Planned revenue'] = tmp8['Quantity']
tmp8 = tmp8.drop('Quantity', axis = 1)

tmp8['Product cost'] = tmp8['Unit']
tmp8 = tmp8.drop('Unit', axis = 1)

tmp8['Quantity'] = tmp8['cost.1']
tmp8 = tmp8.drop('cost.1', axis = 1)

tmp8['Unit cost'] = tmp8['Unit.1']
tmp8 = tmp8.drop('Unit.1', axis = 1)

tmp8['Unit price'] = tmp8['price']
tmp8 = tmp8.drop('price', axis = 1)

tmp8['Gross profit'] = tmp8['Gross']
tmp8 = tmp8.drop('Gross', axis = 1)

tmp8['Unit sale price'] = tmp8['profit']
tmp8 = tmp8.drop('profit', axis = 1)

tmp8 = tmp8.drop(['Unit.2', 'sale', 'price.1'], axis = 1)

tmp8.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17514,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105
17529,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38682,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210
38697,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320
59850,2006,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [659]:
#第一個資料子集
str1 = tmp8
str1.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17514,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105
17529,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38682,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210
38697,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320
59850,2006,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [660]:
#4.4

#完成第二個資料子集

#排除資料溢位處理好的instance (tmp8)
temp = copy.deepcopy(tmp7[~tmp7.index.isin(tmp8.index.values)]) #temp的資料溢位問題已處理完，剩欄位名要修改
temp.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17515,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17516,2004,Mexico,13497.6,14598.08,3841.6,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17517,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17518,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17519,2004,Korea,8433.46,8788.64,2312.8,826.0,2.8,10.64,6120.66,10.210000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


In [661]:
#修改欄位
temp['Retailer country'] = temp['revenue']
temp = temp.drop('revenue', axis = 1)

temp['Revenue'] = temp['Product.3']
temp = temp.drop('Product.3', axis = 1)

temp['Planned revenue'] = temp['cost']
temp = temp.drop('cost', axis = 1)

temp['Product cost'] = temp['Quantity']
temp = temp.drop('Quantity', axis = 1)

temp['Quantity'] = temp['Unit']
temp = temp.drop('Unit', axis = 1)

temp['Unit cost'] = temp['cost.1']
temp = temp.drop('cost.1', axis = 1)

temp['Unit price'] = temp['Unit.1']
temp = temp.drop('Unit.1', axis = 1)

temp['Gross profit'] = temp['price']
temp = temp.drop('price', axis = 1)

temp['Unit sale price'] = temp['Gross']
temp = temp.drop('Gross', axis =1)

temp = temp.drop(['profit', 'Unit.2', 'sale', 'price.1'], axis =1)

temp.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17515,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17516,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Mexico,13497.6,14598.08,3841.6,1372.0,2.8,10.64,9656.02,6.806667
17517,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17518,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17519,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Korea,8433.46,8788.64,2312.8,826.0,2.8,10.64,6120.66,10.210000


In [662]:
str2 = temp
str2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17515,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17516,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Mexico,13497.6,14598.08,3841.6,1372.0,2.8,10.64,9656.02,6.806667
17517,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17518,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17519,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Korea,8433.46,8788.64,2312.8,826.0,2.8,10.64,6120.66,10.210000


In [663]:
#4.5

#資料子集3
temp2 = copy.deepcopy(tmp6[~tmp6.index.isin(tmp7.index.values)])
temp2.head()

#可以看到還要再處理 Retailer country資料溢位問題和欄位名稱修改

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17493,2004,Telephone,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17494,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17495,2004,Telephone,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17496,2004,Telephone,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17497,2004,Telephone,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [664]:
#修改欄位和處理 Retailer country資料溢位問題

temp2['Order method type'] = temp2['Revenue']
temp2 = temp2.drop('Revenue', axis = 1)

temp2.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17493,2004,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17494,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17495,2004,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17496,2004,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17497,2004,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone


In [665]:
temp3 = copy.deepcopy(temp2[temp2['revenue'].isin(['States','Kingdom'])]) #留下有資料溢位問題的資料
temp3['Retailer country'] = temp3['Planned'] + " " + temp3['revenue']
temp3 = temp3.drop(['Planned', 'revenue'], axis = 1)
temp3.head()

,Year,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
17493,2004,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United States
17508,2004,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United Kingdom
17535,2004,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United States
17550,2004,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United Kingdom
17556,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special,United States


In [666]:
#調整後面數字型態的資料的欄位名
temp3['Revenue'] = temp3['Product.3']
temp3 = temp3.drop('Product.3', axis = 1)

temp3['Planned revenue'] = temp3['cost']
temp3 = temp3.drop('cost', axis = 1)

temp3['Product cost'] = temp3['Quantity']
temp3 = temp3.drop('Quantity', axis = 1)

temp3['Quantity'] = temp3['Unit']
temp3 = temp3.drop('Unit', axis = 1)

temp3['Unit cost'] = temp3['cost.1']
temp3 = temp3.drop('cost.1', axis = 1)

temp3['Unit price'] = temp3['Unit.1']
temp3 = temp3.drop('Unit.1', axis = 1)

temp3['Gross profit'] = temp3['price']
temp3 = temp3.drop('price', axis = 1)

temp3['Unit sale price'] = temp3['Gross']
temp3 = temp3.drop('Gross', axis = 1)

temp3 = temp3.drop(['profit', 'Unit.2', 'sale', 'price.1'], axis = 1)

temp3.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17493,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United States,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613
17508,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United Kingdom,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000
17535,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United States,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818
17550,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United Kingdom,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909
17556,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [667]:
str3 = temp3
str3.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17493,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United States,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613
17508,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United Kingdom,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000
17535,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United States,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818
17550,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United Kingdom,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909
17556,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [668]:
#資料子集4

temp2 = temp2[~temp2.index.isin(temp3.index.values)] #排除資料子集3處理好的 instance
temp2.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17494,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17495,2004,Mexico,32224.2,35005.6,9212.0,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17496,2004,Brazil,10268.4,12012.6,3161.2,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17497,2004,Japan,32314.7,35271.6,9282.0,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17498,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone


In [669]:
#因為沒有資料溢位問題，所以修改欄位即可
temp2['Retailer country'] = temp2['Planned']
temp2 = temp2.drop('Planned', axis = 1)

temp2['Revenue'] = temp2['revenue']
temp2 = temp2.drop('revenue', axis = 1)

temp2['Planned revenue'] = temp2['Product.3']
temp2 = temp2.drop('Product.3', axis = 1)

temp2['Product cost'] = temp2['cost']
temp2 = temp2.drop('cost', axis = 1)

temp2['Quantity_buffer'] = temp2['Quantity']
temp2 = temp2.drop('Quantity', axis = 1)

temp2['Quantity'] = temp2['Quantity_buffer']
temp2 = temp2.drop('Quantity_buffer', axis = 1)

temp2['Unit cost'] = temp2['Unit']
temp2 = temp2.drop('Unit', axis = 1)

temp2['Unit price'] = temp2['cost.1']
temp2 = temp2.drop('cost.1', axis = 1)

temp2['Gross profit'] = temp2['Unit.1']
temp2 = temp2.drop('Unit.1', axis = 1)

temp2['Unit sale price'] = temp2['price']
temp2 = temp2.drop('price', axis = 1)

temp2 = temp2.drop(['Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17494,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17495,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Mexico,32224.2,35005.6,9212.0,3290.0,2.8,10.64,23012.18,7.308571
17496,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Brazil,10268.4,12012.6,3161.2,1129.0,2.8,10.64,7107.20,4.128000
17497,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Japan,32314.7,35271.6,9282.0,3315.0,2.8,10.64,23032.65,7.292857
17498,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [670]:
str4 = temp2
str4.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17494,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17495,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Mexico,32224.2,35005.6,9212.0,3290.0,2.8,10.64,23012.18,7.308571
17496,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Brazil,10268.4,12012.6,3161.2,1129.0,2.8,10.64,7107.20,4.128000
17497,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Japan,32314.7,35271.6,9282.0,3315.0,2.8,10.64,23032.65,7.292857
17498,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [671]:
#資料子集5

temp4 = copy.deepcopy(tmp5[~tmp5.index.isin(tmp6.index.values)])
temp4.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2793,2004,Hibernator,Self,0,Inflating,Mat,Telephone,United,States,901517,...,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2794,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Canada,57049.5,61344.1,...,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2795,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Mexico,28879.8,31053.9,...,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2796,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Brazil,19647.8,21126.8,...,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2797,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Japan,423905,470263,...,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [672]:
#合併 Product.2 Order method type.1 Retailer成 Product
temp4['Product'] = temp4['Product.2'] + " " + temp4['Order'] + " " + temp4['method'].astype(str) + " " + temp4['type.1'] + " " + temp4['Retailer']
temp4 = temp4.drop(['Product.2', 'Order', 'method', 'type.1', 'Retailer'], axis = 1)
temp4.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2793,2004,Telephone,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2794,2004,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2795,2004,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2796,2004,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2797,2004,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat


In [673]:
#處理 Order method type資料溢位(只有sales visit會溢位)

temp5 = copy.deepcopy(temp4[temp4['Revenue'] == 'visit'])
temp5.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2814,2004,Sales,visit,United,States,339693,365264.9,198030.0,2870.0,69.00,127.27,141663.200000,118.36,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2815,2004,Sales,visit,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2816,2004,Sales,visit,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2817,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2818,2004,Sales,visit,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat


In [674]:
#temp5的 country和 Revenue合併成 Order method type
temp5['Order method type'] = temp5['country'] + " " + temp5['Revenue']
temp5 = temp5.drop(['country', 'Revenue'], axis = 1)
temp5.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2814,2004,United,States,339693,365264.9,198030.0,2870.0,69.00,127.27,141663.200000,118.36,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2815,2004,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2816,2004,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2817,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2818,2004,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit


In [675]:
#處理 Retailer country資料溢位問題(United States和 United Kingdom)
temp6 = copy.deepcopy(temp5[temp5['revenue'].isin(['States', 'Kingdom'])])

#合併 Planned, revenue成 Retailer country
temp6['Retailer country'] = temp6['Planned'] + " " + temp6['revenue']
temp6 = temp6.drop(['Planned', 'revenue'], axis =1)

#修正欄位名
temp6['Revenue'] = temp6['Product.3']
temp6 = temp6.drop('Product.3', axis = 1)

temp6['Planned revenue'] = temp6['cost']
temp6 = temp6.drop('cost', axis = 1)

temp6['Product cost'] = temp6['Quantity']
temp6 = temp6.drop('Quantity', axis =1)

temp6['Quantity'] = temp6['Unit']
temp6 = temp6.drop('Unit', axis = 1)

temp6['Unit cost'] = temp6['cost.1']
temp6 = temp6.drop('cost.1', axis = 1)

temp6['Unit price'] = temp6['Unit.1']
temp6 = temp6.drop('Unit.1', axis = 1)

temp6['Gross profit'] = temp6['price']
temp6 = temp6.drop('price', axis = 1)

temp6['Unit sale price'] = temp6['Gross']
temp6 = temp6.drop('Gross', axis = 1)

temp6 = temp6.drop(['profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp6.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2814,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,339693,365264.90,198030.0,2870.0,69.00,127.27,141663.20,118.360
2829,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United Kingdom,22961.8,24690.38,13386.0,194.0,69.00,127.27,9575.84,118.360
23982,2005,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,206481,219668.02,94153.3,1726.0,54.55,127.27,112328.08,119.630
23997,2005,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45150,2006,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,236268,264212.52,109093.8,2076.0,52.55,127.27,127174.02,115.818


In [676]:
str5 = temp6
str5.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2814,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,339693,365264.90,198030.0,2870.0,69.00,127.27,141663.20,118.360
2829,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United Kingdom,22961.8,24690.38,13386.0,194.0,69.00,127.27,9575.84,118.360
23982,2005,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,206481,219668.02,94153.3,1726.0,54.55,127.27,112328.08,119.630
23997,2005,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45150,2006,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,United States,236268,264212.52,109093.8,2076.0,52.55,127.27,127174.02,115.818


In [677]:
#資料子集6

temp7 = copy.deepcopy(temp5[~temp5.index.isin(temp6.index.values)]) #排除處理好 Retailer country溢位問題的資料
temp7.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2815,2004,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2816,2004,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2817,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2818,2004,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit
2819,2004,Korea,355384,397464,215487.0,3123.0,69.0,127.27,139896.79,114.701250,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit


In [678]:
#修改欄位名稱即可
temp7['Retailer country'] = temp7['Planned']
temp7 = temp7.drop('Planned', axis = 1)

temp7['Revenue'] = temp7['revenue']
temp7 = temp7.drop('revenue', axis = 1)

temp7['Planned revenue'] = temp7['Product.3']
temp7 = temp7.drop('Product.3', axis = 1)

temp7['Product cost'] = temp7['cost']
temp7 = temp7.drop('cost', axis =1)

temp7['Quantity_buffer'] = temp7['Quantity']
temp7 = temp7.drop('Quantity', axis = 1)
temp7['Quantity'] = temp7['Quantity_buffer']
temp7 = temp7.drop('Quantity_buffer', axis = 1)

temp7['Unit cost'] = temp7['Unit']
temp7 = temp7.drop('Unit', axis = 1)

temp7['Unit price'] = temp7['cost.1']
temp7 = temp7.drop('cost.1', axis = 1)

temp7['Gross profit'] = temp7['Unit.1']
temp7 = temp7.drop('Unit.1', axis = 1)

temp7['Unit sale price'] = temp7['price']
temp7 = temp7.drop('price', axis = 1)

temp7 = temp7.drop(['Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)

temp7.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2815,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000
2816,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778
2817,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2818,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000
2819,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Korea,355384,397464,215487.0,3123.0,69.0,127.27,139896.79,114.701250


In [679]:
str6 = temp7
str6.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2815,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000
2816,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778
2817,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2818,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000
2819,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Sales visit,Korea,355384,397464,215487.0,3123.0,69.0,127.27,139896.79,114.701250


In [680]:
#資料子集7

temp8 = copy.deepcopy(temp4[~temp4.index.isin(temp5.index.values)]) #排除 Product = Hibernator Self 0 Inflating Mat &  Order method type = Sales visit的資料

temp8.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2793,2004,Telephone,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2794,2004,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2795,2004,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2796,2004,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat
2797,2004,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat


In [681]:
temp8['Order method type'] = temp8['country']
temp8 = temp8.drop('country', axis = 1)
temp8.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2793,2004,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2794,2004,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2795,2004,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2796,2004,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2797,2004,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone


In [682]:
#先處理 Retailer country溢位問題
temp9 = copy.deepcopy(temp8[temp8['Planned'].isin(['States', 'Kingdom'])])
temp9['Retailer country'] = temp9['Revenue'] + " " + temp9['Planned']
temp9 = temp9.drop(['Revenue', 'Planned'], axis = 1)
temp9.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
2793,2004,901517,1.06232e+06,575943.0,8347.0,69.0,127.27,325573.61,110.164400,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United States
2808,2004,297037,352665,191199.0,2771.0,69.0,127.27,105838.17,109.579000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United Kingdom
2835,2004,411303,457027,247779.0,3591.0,69.0,127.27,163524.13,115.920833,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United States
2850,2004,317259,349356,189405.0,2745.0,69.0,127.27,127853.73,116.108462,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United Kingdom
2856,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Special,United States


In [683]:
#修改欄位名稱
temp9['Revenue'] = temp9['revenue']
temp9 = temp9.drop('revenue', axis = 1)

temp9['Planned revenue'] = temp9['Product.3']
temp9 = temp9.drop('Product.3', axis = 1)

temp9['Product cost'] = temp9['cost']
temp9 = temp9.drop('cost', axis = 1)

temp9['Quantity_buffer'] = temp9['Quantity']
temp9 = temp9.drop('Quantity', axis = 1)
temp9['Quantity'] = temp9['Quantity_buffer']
temp9 = temp9.drop('Quantity_buffer', axis = 1)

temp9['Unit cost'] = temp9['Unit']
temp9 = temp9.drop('Unit', axis = 1)

temp9['Unit price'] = temp9['cost.1']
temp9 = temp9.drop('cost.1', axis = 1)

temp9['Gross profit'] = temp9['Unit.1']
temp9 = temp9.drop('Unit.1', axis = 1)

temp9['Unit sale price'] = temp9['price']
temp9 = temp9.drop('price', axis = 1)

temp9 = temp9.drop(['Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)

temp9.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2793,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United States,901517,1.06232e+06,575943.0,8347.0,69.0,127.27,325573.61,110.164400
2808,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United Kingdom,297037,352665,191199.0,2771.0,69.0,127.27,105838.17,109.579000
2835,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United States,411303,457027,247779.0,3591.0,69.0,127.27,163524.13,115.920833
2850,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United Kingdom,317259,349356,189405.0,2745.0,69.0,127.27,127853.73,116.108462
2856,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [684]:
str7 = temp9
str7.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2793,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United States,901517,1.06232e+06,575943.0,8347.0,69.0,127.27,325573.61,110.164400
2808,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,United Kingdom,297037,352665,191199.0,2771.0,69.0,127.27,105838.17,109.579000
2835,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United States,411303,457027,247779.0,3591.0,69.0,127.27,163524.13,115.920833
2850,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Web,United Kingdom,317259,349356,189405.0,2745.0,69.0,127.27,127853.73,116.108462
2856,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [685]:
#資料子集8

temp10 = copy.deepcopy(temp8[~temp8.index.isin(temp9.index.values)])
temp10.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2794,2004,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2795,2004,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2796,2004,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2797,2004,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone
2798,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone


In [686]:
#修正欄位名稱
temp10['Retailer country'] = temp10['Revenue']
temp10 = temp10.drop('Revenue', axis = 1)

temp10['Revenue'] = temp10['Planned']
temp10 = temp10.drop('Planned', axis = 1)

temp10['Planned revenue'] = temp10['revenue']
temp10 = temp10.drop('revenue', axis = 1)

temp10['Product cost'] = temp10['Product.3']
temp10 = temp10.drop('Product.3', axis = 1)

buffer = temp10['Quantity']
temp10 = temp10.drop('Quantity', axis = 1)

temp10['Quantity'] = temp10['cost']
temp10 = temp10.drop('cost', axis = 1)

temp10['Unit cost'] = buffer
del buffer

temp10['Unit price'] = temp10['Unit']
temp10 = temp10.drop('Unit', axis = 1)

temp10['Gross profit'] = temp10['cost.1']
temp10 = temp10.drop('cost.1', axis = 1)

temp10['Unit sale price'] = temp10['Unit.1']
temp10 = temp10.drop('Unit.1', axis = 1)

temp10 = temp10.drop(['price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp10.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2794,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000
2795,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000
2796,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000
2797,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833
2798,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [687]:
str8 = temp10
str8.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2794,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000
2795,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000
2796,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000
2797,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833
2798,2004,Camping Equipment,Sleeping Bags,Hibernator Self 0 Inflating Mat,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [688]:
#資料子集9

temp11 = copy.deepcopy(tmp4[~tmp4.index.isin(tmp5.index.values)])
temp11.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
735,2004,TrailChef,Deluxe,Cook,Set,Telephone,United,States,865351,930482,...,254856.69,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
736,2004,TrailChef,Deluxe,Cook,Set,Telephone,Canada,67799.7,72902.6,47831.8,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
737,2004,TrailChef,Deluxe,Cook,Set,Telephone,Mexico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
738,2004,TrailChef,Deluxe,Cook,Set,Telephone,Brazil,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
739,2004,TrailChef,Deluxe,Cook,Set,Telephone,Japan,419827,451426,296183,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [689]:
#合併 Product.2, Order, method, type.1成 Product
temp11['Product'] = temp11['Product.2'] + " " + temp11['Order'] + " " + temp11['method'] + " " + temp11['type.1']
temp11 = temp11.drop(['Product.2', 'Order', 'method', 'type.1'], axis = 1)
temp11.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
735,2004,Telephone,United,States,865351,930482,610494,7173.00,85.11,129.72,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
736,2004,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
737,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
738,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
739,2004,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set


In [690]:
#挑出有 Sales visit的資料
temp12 = copy.deepcopy(temp11[temp11['country'] == 'visit'])
#合併 Retailer, country成 Order method type
temp12['Order method type'] = temp12['Retailer'] + " " + temp12['country']
temp12 = temp12.drop(['Retailer', 'country'], axis = 1)
temp12.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
756,2004,United,States,465791,500849,328609.71,3861.00,85.11,129.72,137181.33,120.64,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
757,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
758,2004,Mexico,365177,392662,257628,3027.00,85.11,129.72,107549.31,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
759,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
760,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit


In [691]:
#處理 Retailer country資料溢位問題(Unites States, United Kingdom)
temp13 = copy.deepcopy(temp12[temp12['Planned'].isin(['States', 'Kingdom'])])
#合併 Revenue, Planned成 Retailer country
temp13['Retailer country'] = temp13['Revenue'] + " " + temp13['Planned']
temp13 = temp13.drop(['Revenue', 'Planned'], axis = 1)
temp13.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
756,2004,465791,500849,328609.71,3861.0,85.11,129.72,137181.33,120.640000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United States
771,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United Kingdom
15309,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United States
15324,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United Kingdom
17808,2004,40004.9,44517.9,16179.10,203.0,79.70,219.30,23825.76,194.446667,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit,United States


In [692]:
#修正欄位名稱
temp13['Revenue'] = temp13['revenue']
temp13 = temp13.drop('revenue', axis = 1)

temp13['Planned revenue'] = temp13['Product.3']
temp13 = temp13.drop('Product.3', axis = 1)

temp13['Product cost'] = temp13['cost']
temp13 = temp13.drop('cost', axis = 1)

temp13['Quantity_buffer']  = temp13['Quantity']
temp13 = temp13.drop('Quantity', axis = 1)
temp13['Quantity'] = temp13['Quantity_buffer']
temp13 = temp13.drop('Quantity_buffer', axis = 1)

temp13['Unit cost'] = temp13['Unit']
temp13 = temp13.drop('Unit', axis = 1)

temp13['Unit price'] = temp13['cost.1']
temp13 = temp13.drop('cost.1', axis = 1)

temp13['Gross profit'] = temp13['Unit.1']
temp13 = temp13.drop('Unit.1', axis = 1)

temp13['Unit sale price'] = temp13['price']
temp13 = temp13.drop('price', axis = 1)

temp13 = temp13.drop(['Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp13.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
756,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United States,465791,500849,328609.71,3861.0,85.11,129.72,137181.33,120.640000
771,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15309,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15324,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17808,2004,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit,United States,40004.9,44517.9,16179.10,203.0,79.70,219.30,23825.76,194.446667


In [693]:
str9 = temp13
str9.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
756,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United States,465791,500849,328609.71,3861.0,85.11,129.72,137181.33,120.640000
771,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15309,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15324,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17808,2004,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit,United States,40004.9,44517.9,16179.10,203.0,79.70,219.30,23825.76,194.446667


In [694]:
#資料子集10

temp14 = copy.deepcopy(temp12[~temp12.index.isin(temp13.index.values)])
temp14.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
757,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
758,2004,Mexico,365177,392662,257628,3027.0,85.11,129.72,107549.31,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
759,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
760,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
761,2004,Korea,522854,562206,368867,4334.0,85.11,129.72,153987.02,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit


In [695]:
#沒有資料溢位，所以修改欄位名稱即可
temp14['Retailer country'] = temp14['Revenue']
temp14 = temp14.drop('Revenue', axis = 1)

temp14['Revenue'] = temp14['Planned']
temp14 = temp14.drop('Planned', axis = 1)

temp14['Planned revenue'] = temp14['revenue']
temp14 = temp14.drop('revenue', axis = 1)

temp14['Product cost'] = temp14['Product.3']
temp14 = temp14.drop('Product.3', axis = 1)

buffer = temp14['Quantity']
temp14 = temp14.drop('Quantity', axis = 1)

temp14['Quantity'] = temp14['cost']
temp14 = temp14.drop('cost', axis = 1)

temp14['Unit cost'] = buffer
del buffer

temp14['Unit price'] = temp14['Unit']
temp14 = temp14.drop('Unit', axis = 1)

temp14['Gross profit'] = temp14['cost.1']
temp14 = temp14.drop('cost.1', axis = 1)

temp14['Unit sale price'] = temp14['Unit.1']
temp14 = temp14.drop('Unit.1', axis = 1)

temp14 = temp14.drop(['price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp14.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
757,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Mexico,365177,392662,257628,3027.0,85.11,129.72,107549.31,120.64
759,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Korea,522854,562206,368867,4334.0,85.11,129.72,153987.02,120.64


In [696]:
str10 = temp14
str10.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
757,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Mexico,365177,392662,257628,3027.0,85.11,129.72,107549.31,120.64
759,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Korea,522854,562206,368867,4334.0,85.11,129.72,153987.02,120.64


In [697]:
#資料子集11
temp15 = copy.deepcopy(temp11[~temp11.index.isin(temp12.index.values)])

#修正 Retailer欄，變成 Order method type
temp15['Order method type'] = temp15['Retailer']
temp15 = temp15.drop('Retailer', axis = 1)
temp15.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
735,2004,United,States,865351,930482,610494,7173.00,85.11,129.72,254856.69,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
736,2004,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
737,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
738,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
739,2004,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone


In [698]:
#處理 Retailer country溢位問題(United States, United Kingdom)
temp16 = copy.deepcopy(temp15[temp15['Revenue'].isin(['States', 'Kingdom'])])
#合併 country, Revenue成 Retailer country
temp16['Retailer country'] = temp16['country'] + " " + temp16['Revenue']
temp16 = temp16.drop(['country', 'Revenue'], axis = 1)
temp16.head()


,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
735,2004,865351,930482,610494,7173.0,85.11,129.72,254856.69,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United States
750,2004,306426,329489,216179,2540.0,85.11,129.72,90246.20,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United Kingdom
777,2004,749657,806080,528874,6214.0,85.11,129.72,220783.42,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United States
792,2004,476649,512524,336270,3951.0,85.11,129.72,140379.03,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United Kingdom
798,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special,United States


In [699]:
#修正欄位
temp16['Revenue'] = temp16['Planned']
temp16 = temp16.drop('Planned', axis = 1)

temp16['Planned revenue'] = temp16['revenue']
temp16 = temp16.drop('revenue', axis = 1)

temp16['Product cost'] = temp16['Product.3']
temp16 = temp16.drop('Product.3', axis = 1)

buffer = temp16['Quantity']
temp16 = temp16.drop('Quantity', axis = 1)

temp16['Quantity'] = temp16['cost']
temp16 = temp16.drop('cost', axis = 1)

temp16['Unit cost'] = buffer
del buffer

temp16['Unit price'] = temp16['Unit']
temp16 = temp16.drop('Unit', axis = 1)

temp16['Gross profit'] = temp16['cost.1']
temp16 = temp16.drop('cost.1', axis = 1)

temp16['Unit sale price'] = temp16['Unit.1']
temp16 = temp16.drop('Unit.1', axis = 1)

temp16 = temp16.drop(['price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp16.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
735,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United States,865351,930482,610494,7173.0,85.11,129.72,254856.69,120.64
750,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United Kingdom,306426,329489,216179,2540.0,85.11,129.72,90246.20,120.64
777,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United States,749657,806080,528874,6214.0,85.11,129.72,220783.42,120.64
792,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United Kingdom,476649,512524,336270,3951.0,85.11,129.72,140379.03,120.64
798,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [700]:
str11 = temp16
str11.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
735,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United States,865351,930482,610494,7173.0,85.11,129.72,254856.69,120.64
750,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United Kingdom,306426,329489,216179,2540.0,85.11,129.72,90246.20,120.64
777,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United States,749657,806080,528874,6214.0,85.11,129.72,220783.42,120.64
792,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United Kingdom,476649,512524,336270,3951.0,85.11,129.72,140379.03,120.64
798,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [701]:
#資料子集12

temp17 = copy.deepcopy(temp15[~temp15.index.isin(temp16.index.values)])
temp17.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
736,2004,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
737,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
738,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
739,2004,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
740,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone


In [702]:
#修正欄位
temp17['Retailer country'] = temp17['country']
temp17 = temp17.drop('country', axis = 1)

buffer = temp17['Revenue']
temp17 = temp17.drop('Revenue', axis = 1)
temp17['Revenue'] = buffer
del buffer

temp17['Planned revenue'] = temp17['Planned']
temp17 = temp17.drop('Planned', axis = 1)

temp17['Product cost'] = temp17['revenue']
temp17 = temp17.drop('revenue', axis = 1)

buffer = temp17['Quantity']
temp17 = temp17.drop(['Quantity'], axis = 1)
temp17['Quantity'] = temp17['Product.3']
temp17 = temp17.drop('Product.3', axis = 1)

temp17['Unit cost'] = temp17['cost']
temp17 = temp17.drop('cost', axis = 1)

temp17['Unit price'] = buffer
del buffer

temp17['Gross profit'] = temp17['Unit']
temp17 = temp17.drop('Unit', axis = 1)

temp17['Unit sale price'] = temp17['cost.1']
temp17 = temp17.drop('cost.1', axis = 1)

temp17 = temp17.drop(['Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp17.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
736,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64
737,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
738,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64
740,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [703]:
str12 = temp17
str12.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
736,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64
737,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
738,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64
740,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [704]:
#資料子集13

temp18 = copy.deepcopy(tmp3[~tmp3.index.isin(tmp4.index.values)])
#合併 Product.2, Order, method
temp18['Product'] = temp18['Product.2'] + " " + temp18['Order'] + " " + temp18['method']
temp18 = temp18.drop(['Product.2', 'Order', 'method'] , axis = 1)
temp18.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
0,2004,Telephone,United,States,315044,437477,158372,66385,2.552857,6.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
1,2004,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.590000,7145.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
2,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
3,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
4,2004,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.590000,91707.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag


In [705]:
#挑出 Sales visit的資料
temp19 = copy.deepcopy(temp18[temp18['Retailer'] == 'visit'])
#合併 type.1, Retailer成 Order method type
temp19['Order method type'] = temp19['type.1'] + " " + temp19['Retailer']
temp19 = temp19.drop(['type.1', 'Retailer'], axis = 1)

temp19.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
21,2004,United,States,181288,222735,88348.1,33799.00,2.72,6.59,92939.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
22,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
23,2004,Mexico,133617,169943,65215.1,25788,2.72,6.59,68402.21,5.723333,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
24,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
25,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit


In [706]:
#處理 Retailer country 資料溢位問題(United States, United Kingdom)
temp20 = copy.deepcopy(temp19[temp19['Revenue'].isin(['States', 'Kingdom'])]) #挑出有 United States, United Kingdom的
#合併country, Revenue成 Retailer country
temp20['Retailer country'] = temp20['country'] + " " + temp20['Revenue']
temp20 = temp20.drop(['country', 'Revenue'], axis = 1)
temp20.head()


,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
21,2004,181288,222735,88348.1,33799.0,2.72,6.59,92939.60,5.652222,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United States
36,2004,11637.3,12250.8,5391.1,1859.0,2.90,6.59,6246.24,6.260000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United Kingdom
315,2004,96725.2,102887,68865.4,4323.0,15.93,23.80,27859.85,22.612000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United States
330,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United Kingdom
609,2004,302116,322549,205344,5872.0,34.97,54.93,96772.39,42.792727,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit,United States


In [707]:
#修正欄位名稱
temp20['Revenue'] = temp20['Planned']
temp20 = temp20.drop('Planned', axis = 1)

temp20['Planned revenue'] = temp20['revenue']
temp20 = temp20.drop('revenue', axis = 1)

temp20['Product cost'] = temp20['Product.3']
temp20 = temp20.drop('Product.3', axis = 1)

buffer = temp20['Quantity']
temp20 = temp20.drop('Quantity', axis = 1)

temp20['Quantity'] = temp20['cost']
temp20 = temp20.drop('cost', axis = 1)

temp20['Unit cost'] = buffer
del buffer

temp20['Unit price'] = temp20['Unit']
temp20 = temp20.drop('Unit', axis = 1)

temp20['Gross profit'] = temp20['cost.1']
temp20 = temp20.drop('cost.1', axis = 1)

temp20['Unit sale price'] = temp20['Unit.1']
temp20 = temp20.drop('Unit.1', axis = 1)

temp20 = temp20.drop(['price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp20.head()


,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
21,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United States,181288,222735,88348.1,33799.0,2.72,6.59,92939.60,5.652222
36,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United Kingdom,11637.3,12250.8,5391.1,1859.0,2.90,6.59,6246.24,6.260000
315,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United States,96725.2,102887,68865.4,4323.0,15.93,23.80,27859.85,22.612000
330,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,2004,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit,United States,302116,322549,205344,5872.0,34.97,54.93,96772.39,42.792727


In [708]:
str13 = temp20
str13.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
21,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United States,181288,222735,88348.1,33799.0,2.72,6.59,92939.60,5.652222
36,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United Kingdom,11637.3,12250.8,5391.1,1859.0,2.90,6.59,6246.24,6.260000
315,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United States,96725.2,102887,68865.4,4323.0,15.93,23.80,27859.85,22.612000
330,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,2004,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit,United States,302116,322549,205344,5872.0,34.97,54.93,96772.39,42.792727


In [709]:
#資料子集14
temp21 = copy.deepcopy(temp19[~temp19.index.isin(temp20.index.values)])
temp21.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
22,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
23,2004,Mexico,133617,169943,65215.1,25788,2.7200,6.59,68402.21,5.723333,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
24,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
25,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
26,2004,Korea,186697,238492,91891.4,36190,2.6975,6.59,94805.19,5.585000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit


In [710]:
#修正欄位名稱
temp21['Retailer country'] = temp21['country']
temp21 = temp21.drop('country', axis = 1)

buffer = temp21['Revenue']
temp21 = temp21.drop('Revenue', axis = 1)
temp21['Revenue'] = buffer
del buffer

temp21['Planned revenue'] = temp21['Planned']
temp21 = temp21.drop('Planned', axis = 1)

temp21['Product cost'] = temp21['revenue']
temp21 = temp21.drop('revenue', axis = 1)

buffer = temp21['Quantity']
temp21 = temp21.drop(['Quantity'], axis = 1)
temp21['Quantity'] = temp21['Product.3']
temp21 = temp21.drop('Product.3', axis = 1)

temp21['Unit cost'] = temp21['cost']
temp21 = temp21.drop('cost', axis = 1)

temp21['Unit price'] = buffer
del buffer

temp21['Gross profit'] = temp21['Unit']
temp21 = temp21.drop('Unit', axis = 1)

temp21['Unit sale price'] = temp21['cost.1']
temp21 = temp21.drop('cost.1', axis = 1)

temp21 = temp21.drop(['Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp21.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
22,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Mexico,133617,169943,65215.1,25788,2.7200,6.59,68402.21,5.723333
24,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Korea,186697,238492,91891.4,36190,2.6975,6.59,94805.19,5.585000


In [711]:
str14 = temp21
str14.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
22,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Mexico,133617,169943,65215.1,25788,2.7200,6.59,68402.21,5.723333
24,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Korea,186697,238492,91891.4,36190,2.6975,6.59,94805.19,5.585000


In [712]:
#資料子集15
temp22 = copy.deepcopy(temp18[~temp18.index.isin(temp19.index.values)])
#修改 type.1成 Order method type
temp22['Order method type'] = temp22['type.1']
temp22 = temp22.drop('type.1', axis = 1)
temp22.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
0,2004,United,States,315044,437477,158372,66385,2.552857,6.59,156672.570,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
1,2004,Canada,13444.7,14313.5,6298.8,2172,2.9,6.590000,7145.88,6.190,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
2,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
3,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
4,2004,Japan,181120,235237,89413.1,35696,2.657,6.590000,91707.18,5.488,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone


In [713]:
#處理 Retailer country資料溢位問題
temp23 = copy.deepcopy(temp22[temp22['country'].isin(['States', 'Kingdom'])])
#合併 Retailer, countrty成 Retailer country
temp23['Retailer country'] = temp23['Retailer'] + " " + temp23['country']
temp23 = temp23.drop(['Retailer', 'country'], axis = 1)
temp23.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,...,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
0,2004,315044,437477,158372,66385,2.552857,6.59,156672.57,5.195714,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United States
15,2004,160227,220996,80307.1,33535,2.576000,6.59,79920.02,5.270000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United Kingdom
42,2004,340867,407124,164590,61779,2.778500,6.59,176276.82,5.838000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United States
57,2004,168290,199446,80612.1,30265,2.775385,6.59,87678.26,5.877692,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United Kingdom
63,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special,United States


In [714]:
#修正欄位名稱
buffer = temp23['Revenue']
temp23 = temp23.drop('Revenue', axis = 1)
temp23['Revenue'] = buffer
del buffer

temp23['Planned revenue'] = temp23['Planned']
temp23 = temp23.drop('Planned', axis = 1)

temp23['Product cost'] = temp23['revenue']
temp23 = temp23.drop('revenue', axis = 1)

buffer = temp23['Quantity']
temp23 = temp23.drop(['Quantity'], axis = 1)
temp23['Quantity'] = temp23['Product.3']
temp23 = temp23.drop('Product.3', axis = 1)

temp23['Unit cost'] = temp23['cost']
temp23 = temp23.drop('cost', axis = 1)

temp23['Unit price'] = buffer
del buffer

temp23['Gross profit'] = temp23['Unit']
temp23 = temp23.drop('Unit', axis = 1)

temp23['Unit sale price'] = temp23['cost.1']
temp23 = temp23.drop('cost.1', axis = 1)

temp23 = temp23.drop(['Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp23.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
0,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United States,315044,437477,158372,66385,2.552857,6.59,156672.57,5.195714
15,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United Kingdom,160227,220996,80307.1,33535,2.576000,6.59,79920.02,5.270000
42,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United States,340867,407124,164590,61779,2.778500,6.59,176276.82,5.838000
57,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United Kingdom,168290,199446,80612.1,30265,2.775385,6.59,87678.26,5.877692
63,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [715]:
str15 = temp23
str15.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
0,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United States,315044,437477,158372,66385,2.552857,6.59,156672.57,5.195714
15,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United Kingdom,160227,220996,80307.1,33535,2.576000,6.59,79920.02,5.270000
42,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United States,340867,407124,164590,61779,2.778500,6.59,176276.82,5.838000
57,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United Kingdom,168290,199446,80612.1,30265,2.775385,6.59,87678.26,5.877692
63,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [716]:
#資料子集16
temp24 = copy.deepcopy(temp22[~temp22.index.isin(temp23.index.values)])
temp24.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
1,2004,Canada,13444.7,14313.5,6298.8,2172,2.9,6.59,7145.88,6.190,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
2,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
3,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
4,2004,Japan,181120,235237,89413.1,35696,2.657,6.59,91707.18,5.488,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
5,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone


In [717]:
#修正欄位名稱
temp24['Retailer country'] = temp24['Retailer']
temp24 = temp24.drop('Retailer', axis = 1)

buffer = temp24['Revenue']
temp24 = temp24.drop('Revenue', axis = 1)

temp24['Revenue'] = temp24['country']
temp24 = temp24.drop('country', axis = 1)

temp24['Planned revenue'] = buffer
del buffer

temp24['Product cost'] = temp24['Planned']
temp24 = temp24.drop('Planned', axis = 1)

buffer = temp24['Quantity']
temp24 = temp24.drop('Quantity', axis = 1)

temp24['Quantity'] = temp24['revenue']
temp24 = temp24.drop('revenue', axis = 1)

temp24['Unit cost'] = temp24['Product.3']
temp24 = temp24.drop('Product.3', axis = 1)

temp24['Unit price'] = temp24['cost']
temp24 = temp24.drop('cost', axis = 1)

temp24['Gross profit'] = buffer
del buffer

temp24['Unit sale price'] = temp24['Unit']
temp24 = temp24.drop('Unit', axis = 1)

temp24 = temp24.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp24.head()


,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
1,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.59,7145.88,6.190
2,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.59,91707.18,5.488
5,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [718]:
str16 = temp24
str16.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
1,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.59,7145.88,6.190
2,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.59,91707.18,5.488
5,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [719]:
#資料子集17
temp25 = copy.deepcopy(tmp2[~tmp2.index.isin(tmp3.index.values)])
temp25.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
147,2004,TrailChef,Canteen,Telephone,United,States,209592,235238,124284,18774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
148,2004,TrailChef,Canteen,Telephone,Canada,8489.36,12129,6408.16,968,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
149,2004,TrailChef,Canteen,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
150,2004,TrailChef,Canteen,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
151,2004,TrailChef,Canteen,Telephone,Japan,107959,124486,65769.7,9935,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [720]:
#合併 Product.2, Order成 Product
temp25['Product'] = temp25['Product.2'] + " " + temp25['Order']
temp25 = temp25.drop(['Product.2', 'Order'], axis = 1)
temp25.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
147,2004,Telephone,United,States,209592,235238,124284,18774,6.62,12.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen
148,2004,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen
149,2004,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen
150,2004,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen
151,2004,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen


In [721]:
#挑出有 Sales visit的資料
temp26 = copy.deepcopy(temp25[temp25['type.1'] == 'visit'])
temp26['Order method type'] = temp26['method'] + " " + temp26['type.1']
temp26 = temp26.drop(['method', 'type.1'], axis  = 1)
temp26.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
168,2004,United,States,58425.6,66045.6,34894,5271,6.62,12.53,23531.5700,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
169,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
170,2004,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.6375,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
171,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
172,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit


In [722]:
#處理 Retailer country資料溢位問題
temp27 = copy.deepcopy(temp26[temp26['country'].isin(['States', 'Kingdom'])])
temp27['Retailer country'] = temp27['Retailer'] + " " + temp27['country']
temp27 = temp27.drop(['Retailer', 'country'], axis = 1)
temp27.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,...,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
168,2004,58425.6,66045.6,34894,5271,6.620,12.53,23531.57,11.486667,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United States
183,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United Kingdom
462,2004,11644.4,12118.3,2814.35,3311,0.850,3.66,8830.01,3.526667,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United States
477,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United Kingdom
1197,2004,176285,216266,87415.3,16359,5.425,13.22,88869.88,11.303750,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Kettle,Sales visit,United States


In [723]:
#修正欄位名稱
buffer = temp27['Revenue']
temp27 = temp27.drop('Revenue', axis = 1)
temp27['Revenue'] = buffer
del buffer

temp27['Planned revenue'] = temp27['Planned']
temp27 = temp27.drop('Planned', axis = 1)

temp27['Product cost'] = temp27['revenue']
temp27 = temp27.drop('revenue', axis = 1)

buffer = temp27['Quantity']
temp27 = temp27.drop(['Quantity'], axis = 1)
temp27['Quantity'] = temp27['Product.3']
temp27 = temp27.drop('Product.3', axis = 1)

temp27['Unit cost'] = temp27['cost']
temp27 = temp27.drop('cost', axis = 1)

temp27['Unit price'] = buffer
del buffer

temp27['Gross profit'] = temp27['Unit']
temp27 = temp27.drop('Unit', axis = 1)

temp27['Unit sale price'] = temp27['cost.1']
temp27 = temp27.drop('cost.1', axis = 1)

temp27 = temp27.drop(['Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp27.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
168,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United States,58425.6,66045.6,34894,5271,6.620,12.53,23531.57,11.486667
183,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,2004,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United States,11644.4,12118.3,2814.35,3311,0.850,3.66,8830.01,3.526667
477,2004,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2004,Camping Equipment,Cooking Gear,TrailChef Kettle,Sales visit,United States,176285,216266,87415.3,16359,5.425,13.22,88869.88,11.303750


In [724]:
str17 = temp27
str17.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
168,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United States,58425.6,66045.6,34894,5271,6.620,12.53,23531.57,11.486667
183,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,2004,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United States,11644.4,12118.3,2814.35,3311,0.850,3.66,8830.01,3.526667
477,2004,Camping Equipment,Cooking Gear,TrailChef Cup,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2004,Camping Equipment,Cooking Gear,TrailChef Kettle,Sales visit,United States,176285,216266,87415.3,16359,5.425,13.22,88869.88,11.303750


In [725]:
#資料子集18
temp28 = copy.deepcopy(temp26[~temp26.index.isin(temp27.index.values)])
temp28.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
169,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
170,2004,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.637500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
171,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
172,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit
173,2004,Korea,79294.1,88273.9,46637.9,7045,6.62,12.53,32656.25,11.545714,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit


In [726]:
#修正欄位名稱
temp28['Retailer country'] = temp28['Retailer']
temp28 = temp28.drop('Retailer', axis = 1)

buffer = temp28['Revenue']
temp28 = temp28.drop('Revenue', axis = 1)

temp28['Revenue'] = temp28['country']
temp28 = temp28.drop('country', axis = 1)

temp28['Planned revenue'] = buffer
del buffer

temp28['Product cost'] = temp28['Planned']
temp28 = temp28.drop('Planned', axis = 1)

buffer = temp28['Quantity']
temp28 = temp28.drop('Quantity', axis = 1)

temp28['Quantity'] = temp28['revenue']
temp28 = temp28.drop('revenue', axis = 1)

temp28['Unit cost'] = temp28['Product.3']
temp28 = temp28.drop('Product.3', axis = 1)

temp28['Unit price'] = temp28['cost']
temp28 = temp28.drop('cost', axis = 1)

temp28['Gross profit'] = buffer
del buffer

temp28['Unit sale price'] = temp28['Unit']
temp28 = temp28.drop('Unit', axis = 1)

temp28 = temp28.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp28.head()


,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
169,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.637500
171,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Korea,79294.1,88273.9,46637.9,7045,6.62,12.53,32656.25,11.545714


In [727]:
str18 = temp28
str18.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
169,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.637500
171,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Sales visit,Korea,79294.1,88273.9,46637.9,7045,6.62,12.53,32656.25,11.545714


In [728]:
#資料子集19

temp29 = copy.deepcopy(temp25[~temp25.index.isin(temp26.index.values)]) #排除已經處理好的有 Sales visit的資料

# 修正 Order method type
temp29['Order method type'] = temp29['method']
temp29 = temp29.drop('method', axis = 1)
temp29.head()


,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
147,2004,United,States,209592,235238,124284,18774,6.62,12.53,85308.180000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
148,2004,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
149,2004,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
150,2004,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
151,2004,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone


In [729]:
#處理 Retailer country資料溢位問題
temp30 = copy.deepcopy(temp29[temp29['Retailer'].isin(['States', 'Kingdom'])])

#合併 type.1 , Retailer 成 Retailer country
temp30['Retailer country'] = temp30['type.1'] + " " + temp30['Retailer']
temp30 = temp30.drop(['type.1', 'Retailer'], axis = 1)

temp30.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
147,2004,209592,235238,124284,18774,6.62,12.53,85308.18,11.427000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United States
162,2004,108404,123934,65478.4,9891,6.62,12.53,42925.27,11.316667,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United Kingdom
189,2004,156644,176335,93163.3,14073,6.62,12.53,63481.21,11.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United States
204,2004,52519.3,57487.6,30372.6,4588,6.62,12.53,22146.74,11.652500,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United Kingdom
210,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Special,United States


In [730]:
#修正欄位名稱
buffer = temp30['Revenue']
temp30 = temp30.drop('Revenue', axis = 1)

temp30['Revenue'] = temp30['country']
temp30 = temp30.drop('country', axis = 1)

temp30['Planned revenue'] = buffer
del buffer

temp30['Product cost'] = temp30['Planned']
temp30 = temp30.drop('Planned', axis = 1)

buffer = temp30['Quantity']
temp30 = temp30.drop('Quantity', axis = 1)

temp30['Quantity'] = temp30['revenue']
temp30 = temp30.drop('revenue', axis = 1)

temp30['Unit cost'] = temp30['Product.3']
temp30 = temp30.drop('Product.3', axis = 1)

temp30['Unit price'] = temp30['cost']
temp30 = temp30.drop('cost', axis = 1)

temp30['Gross profit'] = buffer
del buffer

temp30['Unit sale price'] = temp30['Unit']
temp30 = temp30.drop('Unit', axis = 1)

temp30 = temp30.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp30.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
147,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United States,209592,235238,124284,18774,6.62,12.53,85308.18,11.427000
162,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United Kingdom,108404,123934,65478.4,9891,6.62,12.53,42925.27,11.316667
189,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United States,156644,176335,93163.3,14073,6.62,12.53,63481.21,11.500000
204,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United Kingdom,52519.3,57487.6,30372.6,4588,6.62,12.53,22146.74,11.652500
210,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [731]:
str19 = temp30
str19.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
147,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United States,209592,235238,124284,18774,6.62,12.53,85308.18,11.427000
162,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,United Kingdom,108404,123934,65478.4,9891,6.62,12.53,42925.27,11.316667
189,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United States,156644,176335,93163.3,14073,6.62,12.53,63481.21,11.500000
204,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Web,United Kingdom,52519.3,57487.6,30372.6,4588,6.62,12.53,22146.74,11.652500
210,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [732]:
#資料子集20

temp31 = copy.deepcopy(temp29[~temp29.index.isin(temp30.index.values)]) #排除 United States, United Kingdom的資料
temp31.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
148,2004,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
149,2004,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
150,2004,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
151,2004,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone
152,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone


In [733]:
#修改欄位名稱
temp31['Retailer country'] = temp31['type.1']
temp31 = temp31.drop('type.1', axis = 1)

buffer = temp31['Revenue']
temp31 = temp31.drop('Revenue', axis = 1)

temp31['Revenue'] = temp31['Retailer']
temp31 = temp31.drop('Retailer', axis = 1)

temp31['Planned revenue'] = temp31['country']
temp31 = temp31.drop('country', axis = 1)

temp31['Product cost'] = buffer
del buffer

buffer = temp31['Quantity']
temp31 = temp31.drop('Quantity', axis = 1)

temp31['Quantity'] = temp31['Planned']
temp31 = temp31.drop('Planned', axis = 1)

temp31['Unit cost'] = temp31['revenue']
temp31 = temp31.drop('revenue', axis = 1)

temp31['Unit price'] = temp31['Product.3']
temp31 = temp31.drop('Product.3', axis = 1)

temp31['Gross profit'] = temp31['cost']
temp31 = temp31.drop('cost', axis = 1)

temp31['Unit sale price'] = buffer
del buffer

temp31 = temp31.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp31.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
148,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000
149,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000
150,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000
151,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923
152,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [734]:
str20 = temp31
str20.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
148,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000
149,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000
150,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000
151,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923
152,2004,Camping Equipment,Cooking Gear,TrailChef Canteen,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [735]:
#資料子集21

temp32 = copy.deepcopy(tmp1[~tmp1.index.isin(tmp2.index.values)])
temp32.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2499,2004,Hibernator,Telephone,United,States,968871,1130591,662200,7700,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2500,2004,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2501,2004,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2502,2004,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2503,2004,Hibernator,Telephone,Japan,624589,720054,421744,4904,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [736]:
#修正欄位名稱
temp32['Product'] = temp32['Product.2']
temp32 = temp32.drop('Product.2', axis = 1)
temp32.head()

,Year,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2499,2004,Telephone,United,States,968871,1130591,662200,7700,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2500,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2501,2004,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2502,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2503,2004,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator


In [737]:
#挑出有 Sales visit的資料
temp33 = copy.deepcopy(temp32[temp32['method'] == 'visit'])
#合併 Order, method成 Order method type
temp33['Order method type'] = temp33['Order'] + " " + temp33['method']
temp33 = temp33.drop(['Order', 'method'], axis = 1)
temp33.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2520,2004,United,States,765609,850733,498284,5794,86,146.83,267325.1900,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2521,2004,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2522,2004,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2523,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2524,2004,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit


In [738]:
#處理 Retailer country 資料溢位問題(United States, United Kingdom)
temp34 = copy.deepcopy(temp33[temp33['Retailer'].isin(['States', 'Kingdom'])])

#合併 type.1, Retailer成 Retailer country
temp34['Retailer country'] = temp34['type.1'] + " " + temp34['Retailer']
temp34 = temp34.drop(['type.1', 'Retailer'], axis = 1)
temp34.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
2520,2004,765609,850733,498284,5794,86,146.83,267325.19,134.004667,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States
2535,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom
23688,2005,234082,249024,145856,1696,86,146.83,88225.92,138.020000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States
23703,2005,60452.8,64311.5,37668,438,86,146.83,22784.76,138.020000,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom
44856,2006,589829,616392,361028,4198,86,146.83,228800.88,140.305556,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States


In [739]:
#修正欄位名稱
buffer = temp34['Revenue']
temp34 = temp34.drop('Revenue', axis = 1)

temp34['Revenue'] = temp34['country']
temp34 = temp34.drop('country', axis = 1)

temp34['Planned revenue'] = buffer
del buffer

temp34['Product cost'] = temp34['Planned']
temp34 = temp34.drop('Planned', axis = 1)

buffer = temp34['Quantity']
temp34 = temp34.drop('Quantity', axis = 1)

temp34['Quantity'] = temp34['revenue']
temp34 = temp34.drop('revenue', axis = 1)

temp34['Unit cost'] = temp34['Product.3']
temp34 = temp34.drop('Product.3', axis = 1)

temp34['Unit price'] = temp34['cost']
temp34 = temp34.drop('cost', axis = 1)

temp34['Gross profit'] = buffer
del buffer

temp34['Unit sale price'] = temp34['Unit']
temp34 = temp34.drop('Unit', axis = 1)

temp34 = temp34.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp34.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2520,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,765609,850733,498284,5794,86,146.83,267325.19,134.004667
2535,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23688,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,234082,249024,145856,1696,86,146.83,88225.92,138.020000
23703,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,60452.8,64311.5,37668,438,86,146.83,22784.76,138.020000
44856,2006,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,589829,616392,361028,4198,86,146.83,228800.88,140.305556


In [740]:
str21 = temp34
str21.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2520,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,765609,850733,498284,5794,86,146.83,267325.19,134.004667
2535,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23688,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,234082,249024,145856,1696,86,146.83,88225.92,138.020000
23703,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,60452.8,64311.5,37668,438,86,146.83,22784.76,138.020000
44856,2006,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,589829,616392,361028,4198,86,146.83,228800.88,140.305556


In [741]:
#資料子集22

temp35 = copy.deepcopy(temp33[~temp33.index.isin(temp34.index.values)])
temp35.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2521,2004,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2522,2004,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2523,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2524,2004,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2525,2004,Korea,590417,673950,394740,4590,86,146.83,195677.14,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit


In [742]:
#修改欄位名稱
temp35['Retailer country'] = temp35['type.1']
temp35 = temp35.drop('type.1', axis = 1)

buffer = temp35['Revenue']
temp35 = temp35.drop('Revenue', axis = 1)

temp35['Revenue'] = temp35['Retailer']
temp35 = temp35.drop('Retailer', axis = 1)

temp35['Planned revenue'] = temp35['country']
temp35 = temp35.drop('country', axis = 1)

temp35['Product cost'] = buffer
del buffer

buffer = temp35['Quantity']
temp35 = temp35.drop('Quantity', axis = 1)

temp35['Quantity'] = temp35['Planned']
temp35 = temp35.drop('Planned', axis = 1)

temp35['Unit cost'] = temp35['revenue']
temp35 = temp35.drop('revenue', axis = 1)

temp35['Unit price'] = temp35['Product.3']
temp35 = temp35.drop('Product.3', axis = 1)

temp35['Gross profit'] = temp35['cost']
temp35 = temp35.drop('cost', axis = 1)

temp35['Unit sale price'] = buffer
del buffer

temp35 = temp35.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp35.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2521,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500
2522,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775
2523,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500
2525,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Korea,590417,673950,394740,4590,86,146.83,195677.14,131.7775


In [743]:
str22 = temp35
str22.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2521,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500
2522,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775
2523,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500
2525,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Korea,590417,673950,394740,4590,86,146.83,195677.14,131.7775


In [744]:
#資料子集23

temp36 = copy.deepcopy(temp32[~temp32.index.isin(temp33.index.values)])
#把 Order改成 Order method type
temp36['Order method type'] = temp36['Order']
temp36 = temp36.drop('Order', axis = 1)
temp36.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2499,2004,United,States,968871,1130591,662200,7700,86,146.83,306670.750,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2500,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2501,2004,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2502,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2503,2004,Japan,624589,720054,421744,4904,86,146.83,202845,130.823,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone


In [745]:
#處理 Retailer country資料溢位問題
temp37 = copy.deepcopy(temp36[temp36['type.1'].isin(['States', 'Kingdom'])])

#合併 method, type.1 成 Retailer country
temp37['Retailer country'] = temp37['method'] + " " + temp37['type.1']
temp37 = temp37.drop(['method', 'type.1'], axis = 1)
temp37.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
2499,2004,968871,1130591,662200,7700,86,146.83,306670.75,129.732143,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United States
2514,2004,558875,632397,370402,4307,86,146.83,188472.97,131.777500,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United Kingdom
2541,2004,833381,906528,530964,6174,86,146.83,302417.07,135.356875,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Web,United States
2556,2004,478974,525358,307708,3578,86,146.83,171265.63,134.959167,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Web,United Kingdom
2562,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Special,United States


In [746]:
#修正欄位名稱
buffer = temp37['Revenue']
temp37 = temp37.drop('Revenue', axis = 1)

temp37['Revenue'] = temp37['Retailer']
temp37 = temp37.drop('Retailer', axis = 1)

temp37['Planned revenue'] = temp37['country']
temp37 = temp37.drop('country', axis = 1)

temp37['Product cost'] = buffer
del buffer

buffer = temp37['Quantity']
temp37 = temp37.drop('Quantity', axis = 1)

temp37['Quantity'] = temp37['Planned']
temp37 = temp37.drop('Planned', axis = 1)

temp37['Unit cost'] = temp37['revenue']
temp37 = temp37.drop('revenue', axis = 1)

temp37['Unit price'] = temp37['Product.3']
temp37 = temp37.drop('Product.3', axis = 1)

temp37['Gross profit'] = temp37['cost']
temp37 = temp37.drop('cost', axis = 1)

temp37['Unit sale price'] = buffer
del buffer

temp37 = temp37.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp37.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2499,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United States,968871,1130591,662200,7700,86,146.83,306670.75,129.732143
2514,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United Kingdom,558875,632397,370402,4307,86,146.83,188472.97,131.777500
2541,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United States,833381,906528,530964,6174,86,146.83,302417.07,135.356875
2556,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United Kingdom,478974,525358,307708,3578,86,146.83,171265.63,134.959167
2562,2004,Camping Equipment,Sleeping Bags,Hibernator,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [747]:
str23 = temp37
str23.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2499,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United States,968871,1130591,662200,7700,86,146.83,306670.75,129.732143
2514,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United Kingdom,558875,632397,370402,4307,86,146.83,188472.97,131.777500
2541,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United States,833381,906528,530964,6174,86,146.83,302417.07,135.356875
2556,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United Kingdom,478974,525358,307708,3578,86,146.83,171265.63,134.959167
2562,2004,Camping Equipment,Sleeping Bags,Hibernator,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [748]:
#資料子集24

temp38 = copy.deepcopy(temp36[~temp36.index.isin(temp37.index.values)])
temp38.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2500,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2501,2004,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2502,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2503,2004,Japan,624589,720054,421744,4904,86,146.83,202845,130.823,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2504,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone


In [749]:
#修正欄位名稱
buffer1 = temp38['Revenue']
temp38 = temp38.drop(['Revenue'], axis = 1)
#buffer2 = temp38['Quantity']
temp38 = temp38.drop(['Quantity'], axis = 1)

temp38['Retailer country'] = temp38['method']
temp38 = temp38.drop('method', axis = 1)

temp38['Revenue'] = temp38['type.1']
temp38 = temp38.drop('type.1', axis = 1)

temp38['Planned revenue'] = temp38['Retailer']
temp38 = temp38.drop('Retailer', axis = 1)

temp38['Product cost'] = temp38['country']
temp38 = temp38.drop('country', axis = 1)

temp38['Quantity'] = buffer1
del buffer1

temp38['Unit cost'] = temp38['Planned']
temp38 = temp38.drop('Planned', axis = 1)

temp38['Unit price'] = temp38['revenue']
temp38 = temp38.drop('revenue', axis = 1)

temp38['Gross profit'] = temp38['Product.3']
temp38 = temp38.drop('Product.3', axis = 1)

temp38['Unit sale price'] = temp38['cost']
temp38 = temp38.drop('cost', axis = 1)

temp38 = temp38.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis = 1)
temp38.head()


,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2500,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2501,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550
2502,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,130.823
2504,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [750]:
str24 = temp38
str24.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2500,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2501,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550
2502,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,130.823
2504,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [751]:
# 4.6

df = pd.concat([str1,str2,str3,str4,str5,str6,str7,str8,str9,str10,str11,str12,str13,str14,str15,str16,str17,str18,str19,str20,str21,str22,str23,str24])
df.info()   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18228 entries, 17514 to 44981
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               18228 non-null  int64  
 1   Product line       18228 non-null  object 
 2   Product type       18228 non-null  object 
 3   Product            18228 non-null  object 
 4   Order method type  18228 non-null  object 
 5   Retailer country   18228 non-null  object 
 6   Revenue            6203 non-null   object 
 7   Planned revenue    6203 non-null   object 
 8   Product cost       6203 non-null   object 
 9   Quantity           6203 non-null   object 
 10  Unit cost          6203 non-null   object 
 11  Unit price         6203 non-null   object 
 12  Gross profit       6203 non-null   object 
 13  Unit sale price    6203 non-null   float64
dtypes: float64(1), int64(1), object(12)
memory usage: 2.1+ MB


In [752]:
df.to_csv('data.csv')